<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/KG_Enhanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain langchain-community faiss-cpu kuzu pyvis
%pip install -qU sentence-transformers torch plotly pandas scikit-learn networkx
%pip install -qU torch torchvision
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.

In [30]:
import kuzu
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from sklearn.manifold import TSNE
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tweepy

In [32]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama-3.1-70b-versatile",
    api_key="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"
)

In [34]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm

In [35]:
chain.invoke({"text": "Explain the importance of low latency for LLMs."})

AIMessage(content="Low latency is crucial for Large Language Models (LLMs) as it directly impacts the user experience and the overall performance of the model. Here's why:\n\n1. **Responsiveness**: Low latency ensures that the model responds quickly to user input, making the interaction feel more natural and engaging. When users ask a question or provide input, they expect a prompt response. High latency can lead to a delay in the response, making the interaction feel sluggish and unresponsive.\n2. **Conversational Flow**: LLMs are designed to facilitate conversations, and low latency is essential for maintaining a smooth conversational flow. When the model responds quickly, users can respond in turn, creating a natural back-and-forth dialogue. High latency can disrupt this flow, making the conversation feel stilted and unnatural.\n3. **User Experience**: Low latency contributes to a better overall user experience. When users experience fast and responsive interactions, they are more l

In [36]:
# Initialize Kuzu DB
db = kuzu.Database("cybersecurity_knowledge_graph")
conn = kuzu.Connection(db)

# Create schema for the graph
conn.execute("CREATE NODE TABLE Entity (name STRING, type STRING, PRIMARY KEY (name))")
conn.execute("CREATE REL TABLE Relation (FROM Entity TO Entity, predicate STRING)")

In [37]:
# Cybersecurity-specific websites
websites = [
    "https://www.cisa.gov/uscert/ncas/alerts",
    "https://www.virustotal.com/gui/home/upload",
    "https://attack.mitre.org/",
    "https://www.darkreading.com/",
    "https://threatpost.com/",
]

In [38]:
# Function to scrape and load web content
def scrape_websites(urls):
    documents = []
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text()
        documents.append(text)
    return documents

In [39]:
# Load and process documents
documents = scrape_websites(websites)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text("\n\n".join(documents))

In [40]:
vectorstore = FAISS.from_texts(texts, embeddings)

In [41]:
# Knowledge extraction and graph population
kg_triple_extract_template = """
Extract up to 5 cybersecurity-related knowledge triplets from the text below in the form (subject, predicate, object).
Focus on threats, vulnerabilities, attack techniques, and security measures.
Text: {text}
Triplets:
"""
kg_triple_extract_prompt = PromptTemplate(
    input_variables=["text"],
    template=kg_triple_extract_template,
)

kg_triple_extract_chain = LLMChain(llm=llm, prompt=kg_triple_extract_prompt)

for text in texts:
    triplets = kg_triple_extract_chain.invoke({"text": text})
    for triplet in triplets['text'].split('\n'):
        if triplet.strip():
            try:
                subject, predicate, obj = eval(triplet.strip())
                conn.execute("INSERT INTO Entity (name, type) VALUES ($1, $2) ON CONFLICT DO NOTHING", [subject, "Cybersecurity_Entity"])
                conn.execute("INSERT INTO Entity (name, type) VALUES ($1, $2) ON CONFLICT DO NOTHING", [obj, "Cybersecurity_Entity"])
                conn.execute("INSERT INTO Relation VALUES ($1, $2, $3)", [subject, obj, predicate])
            except:
                print(f"Failed to process triplet: {triplet}")

Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (subject: "Cyber Threats and Advisories", predicate: "is related to", object: "Cybersecurity Best Practices")
Failed to process triplet: 2. (subject: "Industrial Control Systems", predicate: "are vulnerable to", object: "Cyber Threats")
Failed to process triplet: 3. (subject: "Secure .gov websites", predicate: "use", object: "HTTPS")
Failed to process triplet: 4. (subject: "Cyber Issues", predicate: "can be reported through", object: "Report A Cyber Issue")
Failed to process triplet: 5. (subject: "CISA", predicate: "provides", object: "Free Cyber Services")
Failed to process triplet: Note that these triplets are not explicitly mentioned in the text, but can be inferred from the content and structure of the webpage.
Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (CISA, 

<string>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?


Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (subject: CISA, predicate: provides, object: Free Cyber Services)
Failed to process triplet: 2. (subject: CISA, predicate: reports, object: Cyber Issue)
Failed to process triplet: 3. (subject: Cybersecurity Advisory, predicate: is, object: type of Advisory)
Failed to process triplet: 4. (subject: ICS Advisory, predicate: is, object: type of Advisory)
Failed to process triplet: 5. (subject: Shields Up, predicate: is, object: Cybersecurity Alert)
Failed to process triplet: Note that the text does not provide much information about specific threats, vulnerabilities, attack techniques, and security measures, so the extracted triplets are more general and related to CISA's cybersecurity efforts.
Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Brute Force, is a technique of,

<string>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?


Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Threat Actors, Exploit, Multiple Vulnerabilities in Ivanti Connect Secure and Policy Secure Gateways)
Failed to process triplet: 2. (Phobos Ransomware, Uses, #StopRansomware)
Failed to process triplet: 3. (SVR Cyber Actors, Adapt, Tactics for Initial Cloud Access)
Failed to process triplet: 4. (Threat Actor, Leverages, Compromised Account of Former Employee)
Failed to process triplet: 5. (PRC State-Sponsored Actors, Compromise, U.S. Critical Infrastructure)
Failed to process triplet: Note: These triplets are based on the information provided in the text and may not be exhaustive or definitive.
Failed to process triplet: Unfortunately, there is no text to extract from, as the provided text only contains page numbers and navigation links. Please provide the actual text you'd like me to extract from, and I'll be happy to help you with extracting cybersecur

...

Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Reflection Amplification, is a type of, Denial of Service (DoS) attack)
Failed to process triplet: 2. (Resource Hijacking, involves, unauthorized access to system resources)
Failed to process triplet: 3. (Service Stop, can cause, system shutdown or reboot)
Failed to process triplet: 4. (System Shutdown/Reboot, is a type of, Denial of Service (DoS) attack)
Failed to process triplet: 5. (Direct Network Flood, is a type of, Denial of Service (DoS) attack)
Failed to process triplet: Note: These triplets are based on the general understanding of the terms mentioned in the text and may not be explicitly stated in the text itself.
Failed to process triplet: Here are five cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Microsoft SmartScreen, Exploited by, Stealer Campaign)
Failed to process triplet: 2. (Hamster K

<string>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?


Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Bots & automated attacks, are used by, attackers)
Failed to process triplet: 2. (APIs, are vulnerable to, bots & automated attacks)
Failed to process triplet: 3. (AttackIQ, supports, reporting mandates)
Failed to process triplet: 4. (Conti Group, uses, top-notch training)
Failed to process triplet: 5. (Reporting mandates, improve, visibility into adversary behavior)
Failed to process triplet: Note: These triplets are based on the text and may not be exhaustive or definitive, but rather a representation of the information presented in the text.
Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Lyceum APT, targets, Middle-Eastern energy firms & telecoms)
Failed to process triplet: 2. (Lyceum APT, uses, retooled arsenal)
Failed to process triplet: 3. (TALON, raises, survei

<string>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?


Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Cybercriminals, are stepping up their offensive in, attacks against security infrastructure)
Failed to process triplet: 2. (Cybercriminals, are stepping up their offensive in, disinformation campaigns)
Failed to process triplet: 3. (Retail sector, can lock down their online systems to prevent, fraud)
Failed to process triplet: 4. (Bug bounty hunters, are impacted by, COVID-19)
Failed to process triplet: 5. (Social media giants, the government and citizens, are more prepared against, disinformation campaigns)
Failed to process triplet: Note: These triplets are in the form (subject, predicate, object) and focus on threats, vulnerabilities, attack techniques, and security measures.
Failed to process triplet: Here are 5 cybersecurity-related knowledge triplets extracted from the text:
Failed to process triplet: 1. (Mobile, is a prime phishing attack vector,

In [45]:
def get_graph_data():
    nodes_result = conn.execute("MATCH (e:Entity) RETURN e.name")
    edges_result = conn.execute("MATCH (e1:Entity)-[r:Relation]->(e2:Entity) RETURN e1.name, r.predicate, e2.name")

    nodes = [row.getString(0) for row in nodes_result]
    edges = [(row.getString(0), row.getString(1), row.getString(2)) for row in edges_result]

    return nodes, edges

In [46]:
# Enhanced graph visualization using Plotly
def visualize_graph_plotly():
    nodes, edges = get_graph_data()
    G = nx.Graph()

    for node in nodes:
        G.add_node(node[0])

    for edge in edges:
        G.add_edge(edge[0], edge[2], label=edge[1])

    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    for node, adjacencies in G.adjacency():
        node_adjacencies.append(len(adjacencies))
        node_text.append(f'{node}<br># of connections: {len(adjacencies)}')

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Knowledge Graph',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[ dict(
                            text="",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 ) ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    fig.show()

In [47]:
# Embedding visualization
def visualize_embeddings():
    # Get embeddings
    doc_embeddings = [embeddings.embed_query(text.page_content) for text in texts]

    # Reduce dimensionality for visualization
    tsne = TSNE(n_components=3, random_state=42)
    vis_dims = tsne.fit_transform(doc_embeddings)

    # Create a DataFrame for Plotly
    df = pd.DataFrame(vis_dims, columns=['x', 'y', 'z'])
    df['text'] = [text.page_content[:100] + '...' for text in texts]  # Truncate text for readability

    # Create 3D scatter plot
    fig = px.scatter_3d(df, x='x', y='y', z='z', hover_data=['text'],
                        title='Document Embeddings Visualization')
    fig.show()

In [48]:
# Query function
def query_graph(query):
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in docs])

    query_template = """
    As a cybersecurity expert, given the following context and question, provide a detailed and actionable answer:
    Context: {context}
    Question: {question}
    Answer:
    """
    query_prompt = ChatPromptTemplate.from_template(query_template)
    query_chain = query_prompt | llm

    return query_chain.invoke({"context": context, "question": query})

In [49]:
# Main execution
if __name__ == "__main__":
    visualize_graph_plotly()

    # Example queries
    questions = [
        "What are the latest threats targeting the healthcare industry?",
        "Can you provide details on recent ransomware attacks?",
        "What are the most critical vulnerabilities discovered in the last month?",
        "How can organizations protect against phishing attacks?",
        "What are the emerging trends in cybersecurity for financial institutions?"
    ]

    for question in questions:
        answer = query_graph(question)
        print(f"Question: {question}\nAnswer: {answer.content}\n")

    # Close the database connection
    conn.close()

TypeError: 'QueryResult' object is not iterable